In [1]:
!pwd
%matplotlib inline
import sys
sys.executable

/c/Users/shingo/jupyter_notebook/tf_2_work/statistical_test_sample


'C:\\Users\\shingo\\Anaconda3\\envs\\tfgpu20\\python.exe'

# ポケモンを題材に因果推論を実践してみる
- https://tepppei.hatenablog.com/

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('pokemon_ball_data.csv')
df.info()
display(df.head())
display(df.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
Unnamed: 0      100 non-null int64
rarity          100 non-null int64
ball_type       100 non-null object
thrown_balls    100 non-null int64
dtypes: int64(3), object(1)
memory usage: 3.2+ KB


,Unnamed: 0,rarity,ball_type,thrown_balls
0,0,110,super,3
1,1,230,monster,2
2,2,110,super,4
3,3,110,super,1
4,4,120,super,2


,Unnamed: 0,rarity,ball_type,thrown_balls
count,100.000000,100.000,100,100.000000
unique,NaN,NaN,2,NaN
top,NaN,NaN,monster,NaN
freq,NaN,NaN,50,NaN
mean,49.500000,132.900,NaN,5.980000
std,29.011492,72.826,NaN,6.482268
min,0.000000,10.000,NaN,1.000000
25%,24.750000,65.000,NaN,1.000000
50%,49.500000,132.500,NaN,3.500000
75%,74.250000,201.250,NaN,7.250000


## 線形回帰によるモデル化
#### t検定と同じく、ボール投げる数を目的変数:y、ボールタイプを説明変数:x とした線形回帰モデルをスーパーボールとモンスターボールでそれぞれ作成
<br>

#### 結果：
- スーパーボールのモデル:y = 6.74*x + b: 1回投げるとボール6個分の効果がある
- モンスターボールのモデル:y = 5.22*x + b : 1回投げるのにボール5個分の効果がある

<br>

#### t検定と同じく、（重みの）差があまりない

In [3]:
import sklearn
print(sklearn.__version__)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PoissonRegressor
from sklearn.linear_model import TweedieRegressor

df_dummy = pd.get_dummies(df, columns=['ball_type'])
display(df_dummy.head())
df_dummy.info()

0.23.0


,Unnamed: 0,rarity,thrown_balls,ball_type_monster,ball_type_super
0,0,110,3,0,1
1,1,230,2,1,0
2,2,110,4,0,1
3,3,110,1,0,1
4,4,120,2,0,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
Unnamed: 0           100 non-null int64
rarity               100 non-null int64
thrown_balls         100 non-null int64
ball_type_monster    100 non-null uint8
ball_type_super      100 non-null uint8
dtypes: int64(3), uint8(2)
memory usage: 2.7 KB


In [4]:
def glm_plot(X, y, is_plot=False):
    """線形回帰+plot"""
    reg = TweedieRegressor(alpha=0,  # ペナルティ項
                           power=0,  # Normal distribution
                           link='identity',  # link function
                           fit_intercept=False,  # 切片
                           max_iter=300,  # ソルバーの試行回数
                          )
    reg.fit(X, y)
    print('重み:', reg.coef_)
    y_hat = reg.predict(X)
    
    if is_plot:
        # 可視化
        fig = plt.figure(figsize=(6.0, 6.0))
        plt.plot(X, y, "o")
        plt.plot(X, y_hat, "*", color="r")
        plt.xlabel('x (total_bill)'), plt.ylabel('y (tips)')
        plt.xlim(0, 60), plt.ylim(0, 12)
        plt.show()

In [5]:
# dfからnumpy.ndarrayに変換
y = df_dummy['thrown_balls'].values          # 目的変数：y = [y0, y1, y2, ... yi]
X_m = df_dummy['ball_type_monster'].values   # 説明変数：x = [x0, x1, x2, ... xi]
X_s = df_dummy['ball_type_super'].values   # 説明変数：x = [x0, x1, x2, ... xi]

# [x0, x1, x2,..] から [[x0], [x1], [x2]] に変換
#X = X.reshape(len(X),1)
X_m = X_m[:, None]
X_s = X_s[:, None]
print(X_m.shape, X_s.shape)
    
glm_plot(X_m, y)
glm_plot(X_s, y)

(100, 1) (100, 1)
重み: [5.22]
重み: [6.74]


## 線形重回帰によるモデル化（交絡因子の説明変数を増やす）
#### ボール投げる数を目的変数:y、ボールタイプとポケモンの捕獲しやすさを説明変数:x1,x2 とした線形重回帰モデルをスーパーボールとモンスターボールでそれぞれ作成
<br>

#### 結果：
- スーパーボールのモデル:y = 5.3877 * x1 + 0.0153 * x2 + b: 1回投げるとボール5個分の効果がある
- モンスターボールのモデル:y = 2.1026 * x1 + 0.0176 * x2 + b : 1回投げるとボール2個分の効果がある

<br>

#### t検定と違い、（重み）ボール3個分の差があることが言える

In [6]:
# dfからnumpy.ndarrayに変換
y = df_dummy['thrown_balls'].values          # 目的変数：y = [y0, y1, y2, ... yi]
X_m = np.stack([df_dummy['ball_type_monster'].values, df_dummy['rarity'].values], axis=1)
X_s = np.stack([df_dummy['ball_type_super'].values, df_dummy['rarity'].values], axis=1)

print(X_m.shape, X_s.shape)
#print(X_m[:2])

glm_plot(X_m, y)
glm_plot(X_s, y)

(100, 2) (100, 2)
重み: [2.10262779 0.01755277]
重み: [5.38771974 0.01533198]
